In [82]:
%pip install pandas
%pip install plotly
%pip install scipy
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [83]:
import pandas as pd
import scipy

import json
from pathlib import Path
from datetime import date

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [84]:
def colorize(text, color):
    if color == "red":
        color = 196
    elif color == "green":
        color = 45
    elif color == "blue":
        color = 80
    elif color == "yellow":
        color = 226
    elif color == "purple":
        color = 92
    elif color == "cyan":
        color = 123
    elif color == "white":
        color = 15
    elif color == "black":
        color = 0
    else:
        color = 15

    return f"\x1b[38;5;{color}m{text}\x1b[0m"

In [91]:
input_path = Path("../OUTPUT/")

source_widget = widgets.Dropdown(options=[x.name for x in input_path.iterdir() if x.is_dir()], description="Source:")
display(source_widget)

Dropdown(description='Source:', options=('FAA', 'FTT', 'MIDAS'), value='FAA')

In [92]:
metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

# get test metadata
for file in metadata_files:
        metadata = json.load(file.open())
        # display relevant info about each test

        print(colorize(f"Test {file.stem.replace("_metadata", "")}:", "blue"))
        print(f" - Date: {metadata.get("date")}")
        print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
        print(f" - Material: {metadata.get('material_name')}")
        print(f" - Specimen description: {metadata.get("specimen_description")}")
        print(f" - General comments: {metadata.get('comments')}")

Test 19010001:
 - Date: 2019-01-29T14:11:00
 - Heat flux (kw/m^2): 50.0
 - Material: PMMA 29 Jan 2019
 - Specimen description: FTT PMMA 29 Jan 2019
 - General comments: None
Test 19010002:
 - Date: 2019-01-30T11:31:00
 - Heat flux (kw/m^2): 20.0
 - Material: Smoldering Balsa No1
 - Specimen description: Smoldering Balsa No1
 - General comments: None
Test 19010003:
 - Date: 2019-01-30T12:03:00
 - Heat flux (kw/m^2): 20.0
 - Material: Smoldering Balsa No2
 - Specimen description: Smoldering Balsa No2
 - General comments: None
Test 19010004:
 - Date: 2019-01-30T13:47:00
 - Heat flux (kw/m^2): 20.0
 - Material: Smoldering Ipe no1 20 kWm2
 - Specimen description: Smoldering Ipe no1 20 kWm2
 - General comments: None
Test 19010005:
 - Date: 2019-01-30T14:35:00
 - Heat flux (kw/m^2): 20.0
 - Material: Smoldering Balsa No3 20 kWm2
 - Specimen description: Smoldering Balsa No3 20 kWm2
 - General comments: None
Test 19010006:
 - Date: 2019-01-30T14:50:00
 - Heat flux (kw/m^2): 20.0
 - Material: S

In [ ]:
filename_widget = widgets.Dropdown(options=[str(x.stem).replace("_metadata", "") for x in metadata_files], description="Filename:")

data_file_path = Path(filename_widget.value + "_data.csv")
data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

# Select X & Y axis
x_axis_widget = widgets.Dropdown(description='X axis:', options=data.columns)
y_axis_widget = widgets.Dropdown(description='Y axis:', options=data.columns)
sec_y_axis_widget = widgets.Dropdown(description='Sec. Y axis:', options=data.columns)
sec_x_axis_widget = widgets.Dropdown(description='Sec. X axis:', options=data.columns)

fig = make_subplots(specs=[[{"secondary_y": True}]])

def update_data(event=None):

    data_file_path = Path(filename_widget.value + "_data.csv")
    data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

    x_axis_widget.options = data.columns
    y_axis_widget.options = data.columns

    sec_y_axis_widget.options = data.columns
    sec_x_axis_widget.options = data.columns

    plot()

def plot(event=None):

    clear_output()

    primary_box = widgets.Box(children=[x_axis_widget, y_axis_widget])
    secondary_box = widgets.Box(children=[sec_x_axis_widget, sec_y_axis_widget])
    vbox = widgets.VBox([filename_widget, primary_box, secondary_box])

    display(vbox)

    y_data = data[y_axis_widget.value]
    x_data = data[x_axis_widget.value]

    sec_y_data = data[sec_y_axis_widget.value]
    sec_x_data = data[sec_x_axis_widget.value]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if y_axis_widget.value != x_axis_widget.value:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=f"{y_axis_widget.value} vs {x_axis_widget.value}"), secondary_y=False)
    
    if sec_y_axis_widget.value != sec_x_axis_widget.value:
        fig.add_trace(go.Scatter(x=sec_x_data, y=sec_y_data, mode='lines', name=f"{sec_y_axis_widget.value} vs {sec_x_axis_widget.value}"), secondary_y=True)
    
    # https://stackoverflow.com/questions/73178377/how-to-remove-duplicate-legends-in-plotly-python-when-appending-traces-in-a-loop
    # prevent duplicate legend entries

    # names = set()
    # fig.for_each_trace(
    #     lambda trace:
    #         trace.update(showlegend=False)
    #         if (trace.name in names) else names.add(trace.name))

    fig.show()


x_axis_widget.observe(plot, names="value")
y_axis_widget.observe(plot, names="value")
sec_x_axis_widget.observe(plot, names="value")
sec_y_axis_widget.observe(plot, names="value")
filename_widget.observe(update_data, names="value")

plot()

# widgets.Box(children=[filename_widget, x_axis_widget, y_axis_widget])


In [88]:
# print out material info again:
metadata = json.load((input_path / Path(source_widget.value) / Path(filename_widget.value + "_metadata.json")).open())

test_name = filename_widget.value

print(colorize(f"Selected test: {test_name}", "blue"))
print(f" - Date: {metadata.get('date')}")
print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
print(f" - Material: {metadata.get('material_name')}")
print(f" - Specimen description: {metadata.get("specimen_description")}")
print(f" - General comments: {metadata.get('comments')}")

Selected test: 021204A6
 - Date: 2002-12-04T04:38:00
 - Heat flux (kw/m^2): 50.0
 - Material: None
 - Specimen description: None
 - General comments: None
